<a href="https://colab.research.google.com/github/sagivantebi/Face_Recognition/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Create Photos


In [ ]:
import cv2
import os
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode

def capture_image():
    js_code = """
        async function captureImage() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg');
        }
    """
    display(Javascript(js_code))
    data = eval_js('captureImage()')
    binary = b64decode(data.split(',')[1])
    return cv2.imdecode(np.frombuffer(binary, np.uint8), -1)

# Load the Haar cascade for face detection
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
nameID = str(input("Enter Your Name: ")).lower()
path = '/content/drive/My Drive/FaceImages/' + nameID
while os.path.exists(path):
    print("Name Already Taken")
    nameID = str(input("Enter Your Name Again: ")).lower()
    path = '/content/drive/My Drive/FaceImages/' + nameID  # Keep this path consistent

os.makedirs(path)

while count < 20:
    frame = capture_image()
    if frame is None:
        print("Error: Could not read frame.")
        break

    faces = facedetect.detectMultiScale(frame, 1.3, 5)

    for x, y, w, h in faces:
        count += 1
        print(count)
        name = path + '/' + str(count) + '.jpg'  # Use the correct path here
        print("Creating Images......." + name)
        cv2.imwrite(name, frame[y:y+h, x:x+w])
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)

    # cv2_imshow(frame)

# Go and train in the website: https://teachablemachine.withgoogle.com/train/
After that download all the things and store them in the drive.

Then change the pathes accordingly.


#Test Model

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/drive/My Drive/Colab Notebooks/keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("images/saggg/1.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)
